*Note: this notebook was generated by dsflow*

# meteo_bretagne / create_table

## \[1- config\] get task configuration as defined in dag_specs.yaml

In [2]:
import os
import json
import datetime as dt
from pyspark.sql import SparkSession
from dsflow_core.helpers import DsflowContext

spark = SparkSession.builder.getOrCreate()
dsflow = DsflowContext.create()

spark

In [3]:
default_task_specs = """
    {"source_path": "/data/raw/meteo/ds=2017-09-23",
     "sink_path": "/data/tables/meteo/ds=2017-09-23",
     "ds": "2017-09-23"}
    """

In [5]:
task_specs_raw = os.environ.get('TASK_SPECS', default_task_specs)
task_specs = json.loads(task_specs_raw)

# dsflow alerts if something looks wrong
dsflow.validade_task_specs(task_specs)

print(task_specs)

{'source_path': '/data/raw/meteo/ds=2017-09-23', 'sink_path': '/data/tables/meteo/ds=2017-09-23', 'ds': '2017-09-23'}


In [6]:
# Get task variables:
source_path = task_specs["source_path"]
sink_path = task_specs["sink_path"]
ds = task_specs["ds"]  # ds is the execution date

In [5]:
# https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/information/

## \[2- first data check\] using Spark, print a couple lines from source_path

In [6]:
raw_data = spark.read.text(source_path)

## \[3- process data\] convert raw data into a Spark dataframe

In [7]:
dataset = (spark
.read
.options(wholeFile=True, inferSchema=True)
.json(source_path)
)

number of rows in dataset

In [8]:
dataset.count()

1694

In [9]:
# by default, dataframe uses an auto-detected schema
dataset.printSchema()

root
 |-- datasetid: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- ch: string (nullable = true)
 |    |-- cl: string (nullable = true)
 |    |-- cm: string (nullable = true)
 |    |-- cod_tend: long (nullable = true)
 |    |-- coordonnees: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- ctype1: string (nullable = true)
 |    |-- ctype2: string (nullable = true)
 |    |-- ctype3: string (nullable = true)
 |    |-- ctype4: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- dd: long (nullable = true)
 |    |-- etat_sol: string (nullable = true)
 |    |-- ff: double (nullable = true)
 |    |-- hbas: string (nullable = true)
 |    |-- hnuage1: string (nullable = true)
 |    |-- hnuage2: string (nullable = true)
 |    |-- hnuage3: string (nullable = true)
 |    |-- hnuage4: string (nullable = true)
 |    |-- n: string (nullable = true)
 |    |-- nbas: string (nullable = true)
 |    |-- nnuage1: string 

In [10]:
from pyspark.sql.functions import expr

In [11]:
dataset02 = (dataset
  .selectExpr("cast(fields.date as timestamp) as ts",
          "date(cast(fields.date as date)) as record_date",
          "fields.*")
  )

In [12]:
dataset02.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- record_date: date (nullable = true)
 |-- ch: string (nullable = true)
 |-- cl: string (nullable = true)
 |-- cm: string (nullable = true)
 |-- cod_tend: long (nullable = true)
 |-- coordonnees: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- ctype1: string (nullable = true)
 |-- ctype2: string (nullable = true)
 |-- ctype3: string (nullable = true)
 |-- ctype4: string (nullable = true)
 |-- date: string (nullable = true)
 |-- dd: long (nullable = true)
 |-- etat_sol: string (nullable = true)
 |-- ff: double (nullable = true)
 |-- hbas: string (nullable = true)
 |-- hnuage1: string (nullable = true)
 |-- hnuage2: string (nullable = true)
 |-- hnuage3: string (nullable = true)
 |-- hnuage4: string (nullable = true)
 |-- n: string (nullable = true)
 |-- nbas: string (nullable = true)
 |-- nnuage1: string (nullable = true)
 |-- nnuage2: string (nullable = true)
 |-- nnuage3: string (nullable = true)
 |-- nnuage4: s

In [13]:
dataset02.groupby("record_date").count().show()

+-----------+-----+
|record_date|count|
+-----------+-----+
| 2017-01-06|    8|
| 2017-01-27|    8|
| 2017-02-26|    8|
| 2017-01-24|    8|
| 2017-06-29|    8|
| 2017-02-16|    8|
| 2017-07-31|    8|
| 2017-04-09|    8|
| 2017-03-28|    8|
| 2017-02-28|    8|
| 2017-06-30|    8|
| 2017-01-30|    8|
| 2017-07-06|    8|
| 2017-05-11|    8|
| 2017-02-10|    8|
| 2017-04-25|    8|
| 2017-03-19|    8|
| 2017-05-26|    8|
| 2017-01-04|    8|
| 2017-06-28|    8|
+-----------+-----+
only showing top 20 rows



## First 10 rows

In [14]:
dsflow.display(dataset02)

Showing 10 random rows


,ts,record_date,ch,cl,cm,cod_tend,coordonnees,ctype1,ctype2,ctype3,...,tend,tend24,tn12,tx12,type_de_tendance_barometrique,u,vv,w1,w2,ww
0,2017-05-05 21:00:00,2017-05-05,None,None,None,3,"[48.825833, -3.473167]",None,None,None,...,60,-960,None,None,"En baisse ou stationnaire, puis en hausse, ou ...",93,None,None,None,None
1,2017-05-06 06:00:00,2017-05-06,None,37,None,3,"[48.825833, -3.473167]",7,None,None,...,40,-960,283.950000,285.350000,"En baisse ou stationnaire, puis en hausse, ou ...",97,4000,2,None,10
2,2017-06-02 15:00:00,2017-06-02,None,35,None,7,"[48.825833, -3.473167]",6,None,None,...,-10,-210,None,None,En baisse,76,20000,2,None,2
3,2017-05-26 09:00:00,2017-05-26,10,30,25,8,"[48.825833, -3.473167]",4,None,None,...,-40,-740,None,None,"Stationnaire ou en hausse, puis en baisse, ou ...",72,20000,0,None,1
4,2017-07-25 18:00:00,2017-07-25,None,35,22,8,"[48.825833, -3.473167]",6,4,None,...,-140,-340,288.650000,292.950000,"Stationnaire ou en hausse, puis en baisse, ou ...",83,7000,2,2,2
5,2017-04-06 06:00:00,2017-04-06,10,31,21,3,"[48.825833, -3.473167]",8,4,None,...,40,0,281.050000,283.450000,"En baisse ou stationnaire, puis en hausse, ou ...",90,15000,0,0,0
6,2017-04-05 03:00:00,2017-04-05,None,None,None,5,"[48.825833, -3.473167]",None,None,None,...,-10,570,None,None,"En baisse, puis en hausse, la pression atmosph...",88,None,None,None,None
7,2017-04-22 12:00:00,2017-04-22,None,None,None,8,"[48.825833, -3.473167]",None,None,None,...,-90,-430,None,None,"Stationnaire ou en hausse, puis en baisse, ou ...",77,None,None,None,None
8,2017-06-11 09:00:00,2017-06-11,None,35,24,1,"[48.825833, -3.473167]",6,3,None,...,50,40,None,None,"En hausse, puis stationnaire, ou en hausse, pu...",82,6000,2,None,1
9,2017-06-12 21:00:00,2017-06-12,None,None,None,3,"[48.825833, -3.473167]",None,None,None,...,20,230,None,None,"En baisse ou stationnaire, puis en hausse, ou ...",87,None,None,None,None


In [15]:
dsflow.display(dataset02.describe())

Showing 10 random rows


,summary,ch,cl,cm,cod_tend,ctype1,ctype2,ctype3,ctype4,date,...,tend,tend24,tn12,tx12,type_de_tendance_barometrique,u,vv,w1,w2,ww
0,count,540,992,704,1694,950,507,146,17,1694,...,1694,1689,424,424,1694,1694,981,1008,225,990
1,mean,13.974074074074075,34.788306451612904,23.056818181818183,4.38370720188902,7.393684210526316,3.777120315581854,2.6506849315068495,2.235294117647059,None,...,-0.7083825265643447,-9.727649496743636,283.13136792452826,287.0662735849058,None,81.22136953955136,14790.876656472987,1.9583333333333333,2.28,11.217171717171718
2,stddev,10.072615481046181,4.780293422129401,7.206931947629506,2.6803071205273765,8.014657417164052,2.3303355182865926,2.8323073711860487,3.881517289744854,None,...,129.96557757037158,666.3523979084362,4.13901542826014,4.684761704459992,None,12.003936803107045,8369.626868897245,1.8697003886178163,1.6110112174833369,20.653927961123127
3,min,10,30,20,0,0,0,0,0,2017-01-01T01:00:00+01:00,...,-780,-2230,270.050000,274.350000,En baisse,35,100,0,0,0
4,max,60,62,61,8,9,9,9,9,2017-07-31T23:00:00+02:00,...,830,2910,292.650000,302.250000,"Stationnaire, la pression atmosphérique est la...",99,9000,9,7,89


## \[4- write on disk\] write down the output to disk

In [ ]:
final_dataset = dataset02

In [19]:
(final_dataset
 .write
 .mode("overwrite")
 .parquet(sink_path)
)

In [7]:
dsflow.validade_task_output(task_specs)

'not implemented'